In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = "https://fr.trustpilot.com/categories"

In [3]:
req = requests.get(url)
try:
    soup = BeautifulSoup(req.text, "lxml")
except:
    soup = BeautifulSoup(req.text, "html.parser")

In [4]:
print(soup)

<!DOCTYPE html>
<html lang="fr-FR"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link href="https://cdn.trustpilot.net/brand-assets/1.5.0/favicons/favicon.ico" rel="shortcut icon" type="image/x-icon"/><link href="/manifest.json" rel="manifest"/><meta content="Trustpilot" name="application-name"/><meta content="#000032" name="theme-color"/><link href="https://cdn.trustpilot.net/brand-assets/1.5.0/favicons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/><link href="https://cdn.trustpilot.net/brand-assets/1.5.0/favicons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/><link href="https://cdn.trustpilot.net/brand-assets/1.5.0/favicons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link color="#00b67a" href="https://cdn.trustpilot.net/brand-assets/1.9.0/favicons/safari-pinned-tab.svg" rel="mask-icon"/><meta content="Trustpilot" name="apple-mobile-web-app-title"/><meta content="#000032" name="msap

In [5]:
big_categories = {"fr_name" : [], "link": []}
sub_categories = {"fr_name" : [], "link": []}


section = soup.find("section", class_="categories_rightColumn__x8o71")
divs = section.find_all("div", class_="categories_subCategory__3OxUx")

for div in divs:
    # getting header of big category
    header3 = div.find("h3", class_="categories_subCategoryHeader__3Bd4c")
    
    # getting fr name, link
    name = header3.find("span").text
    link = header3.find("a")["href"]
    link = link.split("/")[-1]
    big_categories["fr_name"].append(name)
    big_categories["link"].append(link)
    
    # getting div of sub categories
    sub_div = div.find("div", class_="categories_subCategoryList__1FB-L")
    every_sub_category_div = sub_div.find_all("div", class_="categories_subCategoryItem__2Qwj8")
    
    # in each sub category we'll find fr name, link
    for sub_category_div in every_sub_category_div:
        sub_name = sub_category_div.find("span").text
        sub_link = sub_category_div.find("a")["href"]
        sub_link = link + "/" + sub_link.split("/")[-1]
        
        sub_categories["fr_name"].append(sub_name)
        sub_categories["link"].append(sub_link)        

    
df_big = pd.DataFrame(data=big_categories)
df_sub = pd.DataFrame(data=sub_categories)
df_big["category"] = 0
df_sub["category"] = 1

In [6]:
df_big.head()

,fr_name,link,category
0,"Aliments, boissons & tabac",food_beverages_tobacco,0
1,Animaux,animals_pets,0
2,Argent & assurance,money_insurance,0
3,Beauté & bien-être,beauty_wellbeing,0
4,Construction & fabrication,construction_manufactoring,0


In [7]:
display(df_sub.head())
df_sub.shape

,fr_name,link,category
0,Agriculture & production,food_beverages_tobacco/agriculture_produce,1
1,Bière & vin,food_beverages_tobacco/beer_wine,1
2,Boissons & alcool,food_beverages_tobacco/beverages_liquor,1
3,Bonbons & chocolat,food_beverages_tobacco/candy_chocolate,1
4,Boulangerie & pâtisserie,food_beverages_tobacco/bakery_pastry,1


(182, 3)

In [8]:
sub_sub_categories = {"fr_name": [], "link": []}

url = "https://fr.trustpilot.com/categories/"

size = len(df_sub)
i = 0

while i < size:
    sub_link = df_sub["link"][i].split("/")[-1]
    req = requests.get(url + sub_link)
    try:
        soup = BeautifulSoup(req.text, "lxml")
    except:
        soup = BeautifulSoup(req.text, "html.parser")

    ul = soup.find("ul", class_="styles_subCategoriesList__1YKDz")
    try:
        lis = ul.find_all("li")
    except:
        print(i, url + sub_link)
    for li in lis:
        a_balise = li.find("a")
        if a_balise["href"] == "#":
            continue
        link = df_sub["link"][i] + "/" + a_balise["href"].split("/")[-1]
        name = a_balise.text.split('(')[0]
        name = name[0:len(name)-1]
        
        sub_sub_categories["fr_name"].append(name)
        sub_sub_categories["link"].append(link)
        
    i += 1

df_sub_sub = pd.DataFrame(data=sub_sub_categories)
df_sub_sub["category"] = 2
df_sub_sub.head()

21 https://fr.trustpilot.com/categories/accounting_tax
22 https://fr.trustpilot.com/categories/real_estate
23 https://fr.trustpilot.com/categories/investments_wealth
24 https://fr.trustpilot.com/categories/credit_debt_services
25 https://fr.trustpilot.com/categories/wellness_spa
26 https://fr.trustpilot.com/categories/personal_care
27 https://fr.trustpilot.com/categories/cosmetics_makeup
28 https://fr.trustpilot.com/categories/salons_clinics
29 https://fr.trustpilot.com/categories/hair_care_styling
30 https://fr.trustpilot.com/categories/tattoos_piercings
31 https://fr.trustpilot.com/categories/yoga_meditation
32 https://fr.trustpilot.com/categories/architects_engineers
33 https://fr.trustpilot.com/categories/contractors_consultants
34 https://fr.trustpilot.com/categories/factory_equipment
35 https://fr.trustpilot.com/categories/manufacturing
36 https://fr.trustpilot.com/categories/industrial_supplies
37 https://fr.trustpilot.com/categories/garden_landscaping
38 https://fr.trustpilot.c

,fr_name,link,category
0,Agriculteur,food_beverages_tobacco/agriculture_produce/cro...,2
1,Coopérative agricole,food_beverages_tobacco/agriculture_produce/agr...,2
2,Ferme,food_beverages_tobacco/agriculture_produce/farm,2
3,Fournisseur de services agricoles,food_beverages_tobacco/agriculture_produce/agr...,2
4,Grossiste de produits,food_beverages_tobacco/agriculture_produce/pro...,2


In [9]:
df_sub_sub.tail()

,fr_name,link,category
1617,Bureau de change,vehicles_transportation/cars_trucks/currency_e...,2
1618,Institution financière,vehicles_transportation/cars_trucks/financial_...,2
1619,Service de crypto-monnaie,vehicles_transportation/cars_trucks/cryptocurr...,2
1620,Service de remise en espèces,vehicles_transportation/cars_trucks/cashback_p...,2
1621,Service de transfert d'argent,vehicles_transportation/cars_trucks/money_tran...,2


In [10]:
print(df_sub_sub.shape)
print(len(df_sub_sub["link"].unique()))
print(len(df_sub_sub["fr_name"].unique()))

(1622, 3)
1622
173


In [11]:
df_sub_sub["fr_name"].value_counts()

Banque du secteur privé                162
Service de transfert d'argent          162
Service de crypto-monnaie              162
Banque d'épargne                       162
Service de remise en espèces           162
                                      ... 
Compagnie d'assurance sportive           1
Magasin de cigarettes électroniques      1
Salon de thé                             1
Magasin de produits gourmets             1
Bar à bières                             1
Name: fr_name, Length: 173, dtype: int64

In [12]:
print(df_sub_sub[df_sub_sub["fr_name"] == "Maçon"]["link"][423])
print(df_sub_sub[df_sub_sub["fr_name"] == "Maçon"]["link"][1072])

KeyError: 423

In [ ]:
print(df_sub.shape)
print(len(df_sub["link"].unique()))
print(len(df_sub["fr_name"].unique()))

In [ ]:
df_sub["fr_name"].value_counts()

In [ ]:
print(df_sub[df_sub["fr_name"] == "Impression & design graphique"]["link"])


In [ ]:
print(df_big.shape)
print(len(df_big["link"].unique()))
print(len(df_big["fr_name"].unique()))

In [ ]:
df = pd.concat([df_big, df_sub, df_sub_sub], ignore_index=True)

In [ ]:
df.shape

In [ ]:
df["category"].value_counts()

In [ ]:
df.to_csv("categories.csv", index=False, sep=";")

In [ ]:
df = pd.read_csv("categories.csv", sep=";")
df.head()